# ResNet-like 1D Neural Network

This note book will be used to test resnet inspired neural network on each of the 3 ontologies.

In [10]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
import yaml

In [11]:
print("TensorFlow v" + tf.__version__)
print("Numpy v" + np.__version__)

TensorFlow v2.15.0
Numpy v1.26.4


In [12]:
config_path = './config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

In [13]:
USE_75_PERCENT_DATA = config['use_75_percent_datasets']
partial_dataset_prefix = '75percent_' if USE_75_PERCENT_DATA else ''
BP_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_BiologicalProcesses.pkl")
CC_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_CellularComponent.pkl")
MF_train_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_embeddings_MolecularFunction.pkl")
BP_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_BiologicalProcesses.pkl")
CC_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_CellularComponent.pkl")
MF_label_df = pd.read_pickle(f"{config['directories']['preprocessed_data']}/{partial_dataset_prefix}train_labels_MolecularFunction.pkl")

In [14]:
train_data_dict = {
    'Biological Processes': [BP_train_df, BP_label_df],
    'Cellular Component': [CC_train_df, CC_label_df],
    'Molecular Function': [MF_train_df, MF_label_df]
}

In [15]:
num_labels = 1500
num_folds = config['num_folds']

In [16]:
BATCH_SIZE = config['batch_size']

In [17]:
from tensorflow.keras import layers, models, Input

def resnet1d_block(input_tensor, in_channels, out_channels, stride=1, downsample=None):
    x = input_tensor
    identity = x

    # First convolution
    x = layers.Conv1D(out_channels, kernel_size=3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second convolution
    x = layers.Conv1D(out_channels, kernel_size=3, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    # Downsample if necessary
    if downsample is not None:
        identity = downsample(input_tensor)

    # Add identity
    x = layers.add([x, identity])
    x = layers.ReLU()(x)

    return x

def ResNet1D(input_shape=(None, 1), num_classes=1500):
    inputs = Input(shape=input_shape)

    # Initial conv and max-pooling
    x = layers.Conv1D(64, kernel_size=7, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool1D(pool_size=3, strides=2, padding='same')(x)

    # Utility function to apply ResNet blocks
    def make_layer(x, in_channels, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = models.Sequential([
                layers.Conv1D(out_channels, kernel_size=1, strides=stride, use_bias=False),
                layers.BatchNormalization(),
            ])

        x = resnet1d_block(x, in_channels, out_channels, stride, downsample)
        for _ in range(1, blocks):
            x = resnet1d_block(x, out_channels, out_channels)
        return x

    # ResNet layers
    x = make_layer(x, 64, 64, 2)
    x = make_layer(x, 64, 128, 2, stride=2)
    x = make_layer(x, 128, 256, 2, stride=2)
    x = make_layer(x, 256, 512, 2, stride=2)

    # Global average pooling and dense layer
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes)(x)

    # Create model
    model = models.Model(inputs=inputs, outputs=outputs)

    return model


In [22]:
def train_model(get_model, dataset_name, model_name, data, num_folds, BATCH_SIZE, loss_function='binary_crossentropy'):
    train, label = data

    best_f1 = 0
    print('=======================================================================')
    print(f'Training for {dataset_name}')
    
    model_root_path = f'{config["directories"]["models"]}/{model_name}'

    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1

    for train_fold, test_fold in kfold.split(train, label):
        print(f'Training for fold {fold_no} ...')
        
        loss_fn = loss_function

        model = get_model()
        model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
          loss=loss_fn,
          metrics=['binary_accuracy',
                tf.keras.metrics.AUC(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                ]
          )

        history = model.fit(
            train.iloc[train_fold], label.iloc[train_fold],
            validation_data=(train.iloc[test_fold], label.iloc[test_fold]),
            batch_size=BATCH_SIZE,
            epochs=5
        )

        scores = model.evaluate(train.iloc[test_fold], label.iloc[test_fold], verbose=0)
        precision = scores[3]
        recall = scores[4]
        F1_score = 2 * precision * recall / (precision + recall)
        print(f'Score for fold {fold_no}: F1 score of {F1_score}; {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')

        if F1_score > best_f1:
            best_f1 = F1_score
            
            if dataset_name == 'Biological Processes':
              tf.keras.models.save_model(
                  model,
                  f'{model_root_path}/best_BP_model',
              )
              print(f'Current best model for Biological Processes has an F1 score of {F1_score}')

            elif dataset_name == 'Molecular Function':
              tf.keras.models.save_model(
                  model,
                  f'{model_root_path}/best_MF_model',
              )
              print(f'Current best model for Molecular Function has an F1 score of {F1_score}')
            else:
              tf.keras.models.save_model(
                  model,
                  f'{model_root_path}/best_CC_model',
              )
              print(f'Current best model for Cellular Component has an F1 score of {F1_score}')

        fold_no += 1

In [23]:
def train_on_datasets(get_model, model_name, loss_function=None):
    for dataset in train_data_dict:
        dataset_name = dataset
        data = train_data_dict[dataset]
        train_model(get_model, dataset_name, model_name, data, 10, BATCH_SIZE, loss_function=loss_function)

In [25]:
train_on_datasets(lambda: ResNet1D(input_shape=(1024, 1), num_classes=1500), 'ResNet1D_BCE', 'binary_crossentropy')

Training for Biological Processes
Training for fold 1 ...
Epoch 1/5
147/147 [==============================] - 35s 88ms/step - loss: 0.3108 - binary_accuracy: 0.9678 - auc_31: 0.6845 - precision_31: 0.3094 - recall_31: 0.1058 - val_loss: 0.4361 - val_binary_accuracy: 0.9661 - val_auc_31: 0.6051 - val_precision_31: 0.2575 - val_recall_31: 0.0999
Epoch 2/5
147/147 [==============================] - 9s 59ms/step - loss: 0.2907 - binary_accuracy: 0.9706 - auc_31: 0.6944 - precision_31: 0.4322 - recall_31: 0.0979 - val_loss: 0.2910 - val_binary_accuracy: 0.9698 - val_auc_31: 0.6878 - val_precision_31: 0.3888 - val_recall_31: 0.1002
Epoch 3/5
147/147 [==============================] - 9s 58ms/step - loss: 0.2844 - binary_accuracy: 0.9713 - auc_31: 0.6969 - precision_31: 0.4859 - recall_31: 0.0863 - val_loss: 0.2962 - val_binary_accuracy: 0.9686 - val_auc_31: 0.6967 - val_precision_31: 0.3561 - val_recall_31: 0.1236
Epoch 4/5
147/147 [==============================] - 8s 57ms/step - loss: 0.2

INFO:tensorflow:Assets written to: ./models/ResNet1D_BCE/best_BP_model/assets


Current best model for Biological Processes has an F1 score of 0.15166213726311673
Training for fold 2 ...
Epoch 1/5
147/147 [==============================] - 17s 61ms/step - loss: 0.3075 - binary_accuracy: 0.9697 - auc_32: 0.6733 - precision_32: 0.3644 - recall_32: 0.0879 - val_loss: 0.2909 - val_binary_accuracy: 0.9719 - val_auc_32: 0.6797 - val_precision_32: 0.6121 - val_recall_32: 0.0845
Epoch 2/5
147/147 [==============================] - 8s 57ms/step - loss: 0.2877 - binary_accuracy: 0.9718 - auc_32: 0.6834 - precision_32: 0.5293 - recall_32: 0.0742 - val_loss: 0.2935 - val_binary_accuracy: 0.9714 - val_auc_32: 0.6773 - val_precision_32: 0.5277 - val_recall_32: 0.1084
Epoch 3/5
147/147 [==============================] - 8s 56ms/step - loss: 0.2813 - binary_accuracy: 0.9723 - auc_32: 0.6880 - precision_32: 0.5991 - recall_32: 0.0780 - val_loss: 0.2845 - val_binary_accuracy: 0.9717 - val_auc_32: 0.6873 - val_precision_32: 0.5755 - val_recall_32: 0.0881
Epoch 4/5
147/147 [=========

INFO:tensorflow:Assets written to: ./models/ResNet1D_BCE/best_BP_model/assets


Current best model for Biological Processes has an F1 score of 0.16929171355113262
Training for fold 3 ...
Epoch 1/5
147/147 [==============================] - 17s 61ms/step - loss: 0.2871 - binary_accuracy: 0.9703 - auc_33: 0.6925 - precision_33: 0.4029 - recall_33: 0.0848 - val_loss: 0.2665 - val_binary_accuracy: 0.9731 - val_auc_33: 0.6985 - val_precision_33: 0.6827 - val_recall_33: 0.0810
Epoch 2/5
147/147 [==============================] - 8s 56ms/step - loss: 0.2722 - binary_accuracy: 0.9721 - auc_33: 0.7023 - precision_33: 0.5924 - recall_33: 0.0793 - val_loss: 0.2577 - val_binary_accuracy: 0.9726 - val_auc_33: 0.7112 - val_precision_33: 0.5527 - val_recall_33: 0.1310
Epoch 3/5
147/147 [==============================] - 8s 56ms/step - loss: 0.2647 - binary_accuracy: 0.9720 - auc_33: 0.7094 - precision_33: 0.5818 - recall_33: 0.0763 - val_loss: 0.2983 - val_binary_accuracy: 0.9656 - val_auc_33: 0.7052 - val_precision_33: 0.3201 - val_recall_33: 0.1994
Epoch 4/5
147/147 [=========

...

Current best model for Molecular Function has an F1 score of 0.20112250977022222
Training for fold 2 ...
Epoch 1/5
98/98 [==============================] - 14s 66ms/step - loss: 0.3341 - binary_accuracy: 0.9670 - auc_52: 0.6714 - precision_52: 0.3769 - recall_52: 0.1134 - val_loss: 0.3080 - val_binary_accuracy: 0.9704 - val_auc_52: 0.6713 - val_precision_52: 0.6048 - val_recall_52: 0.0535
Epoch 2/5
98/98 [==============================] - 6s 58ms/step - loss: 0.3014 - binary_accuracy: 0.9704 - auc_52: 0.6901 - precision_52: 0.6246 - recall_52: 0.0945 - val_loss: 0.2963 - val_binary_accuracy: 0.9705 - val_auc_52: 0.6865 - val_precision_52: 0.5383 - val_recall_52: 0.1403
Epoch 3/5
98/98 [==============================] - 6s 57ms/step - loss: 0.2964 - binary_accuracy: 0.9704 - auc_52: 0.6933 - precision_52: 0.6186 - recall_52: 0.0981 - val_loss: 0.3006 - val_binary_accuracy: 0.9713 - val_auc_52: 0.6805 - val_precision_52: 0.6507 - val_recall_52: 0.1030
Epoch 4/5
98/98 [===================

INFO:tensorflow:Assets written to: ./models/ResNet1D_BCE/best_MF_model/assets


Current best model for Molecular Function has an F1 score of 0.21115643328708691
Training for fold 3 ...
Epoch 1/5
98/98 [==============================] - 15s 67ms/step - loss: 0.3296 - binary_accuracy: 0.9652 - auc_53: 0.6912 - precision_53: 0.3380 - recall_53: 0.1390 - val_loss: 0.3222 - val_binary_accuracy: 0.9679 - val_auc_53: 0.6833 - val_precision_53: 0.3870 - val_recall_53: 0.0667
Epoch 2/5
98/98 [==============================] - 6s 58ms/step - loss: 0.3115 - binary_accuracy: 0.9692 - auc_53: 0.6997 - precision_53: 0.4912 - recall_53: 0.1229 - val_loss: 0.3101 - val_binary_accuracy: 0.9703 - val_auc_53: 0.6968 - val_precision_53: 0.5856 - val_recall_53: 0.1265
Epoch 3/5
98/98 [==============================] - 6s 56ms/step - loss: 0.2965 - binary_accuracy: 0.9698 - auc_53: 0.7034 - precision_53: 0.5409 - recall_53: 0.1094 - val_loss: 0.3023 - val_binary_accuracy: 0.9696 - val_auc_53: 0.6974 - val_precision_53: 0.5394 - val_recall_53: 0.1063
Epoch 4/5
98/98 [===================

INFO:tensorflow:Assets written to: ./models/ResNet1D_BCE/best_MF_model/assets


Current best model for Molecular Function has an F1 score of 0.23434841765448286
Training for fold 4 ...
Epoch 1/5
98/98 [==============================] - 14s 65ms/step - loss: 0.3041 - binary_accuracy: 0.9669 - auc_54: 0.7003 - precision_54: 0.3410 - recall_54: 0.0839 - val_loss: 0.2762 - val_binary_accuracy: 0.9706 - val_auc_54: 0.7068 - val_precision_54: 0.7568 - val_recall_54: 0.0498
Epoch 2/5
98/98 [==============================] - 6s 58ms/step - loss: 0.2784 - binary_accuracy: 0.9698 - auc_54: 0.7143 - precision_54: 0.5551 - recall_54: 0.0901 - val_loss: 0.2676 - val_binary_accuracy: 0.9713 - val_auc_54: 0.7173 - val_precision_54: 0.7487 - val_recall_54: 0.0821
Epoch 3/5
98/98 [==============================] - 6s 57ms/step - loss: 0.2695 - binary_accuracy: 0.9702 - auc_54: 0.7243 - precision_54: 0.5796 - recall_54: 0.1048 - val_loss: 0.2643 - val_binary_accuracy: 0.9701 - val_auc_54: 0.7237 - val_precision_54: 0.6127 - val_recall_54: 0.0403
Epoch 4/5
98/98 [===================